In [ ]:
GIT_ROOT_LINES = !git rev-parse --show-toplevel
WORK_DIR = GIT_ROOT_LINES[0]
%cd $WORK_DIR
%load_ext autoreload
%autoreload 2

In [1]:
import pyvista as pv
import ipywidgets as widgets
from IPython.display import display
import os.path as osp
import motionblender.lib.pv_editor as pv_editor

from glob import glob
from copy import deepcopy
import os.path as osp
import torch
import numpy as np
from motionblender.lib.misc import dump_json, load_cpkl, dump_cpkl
import motionblender.lib.convert_utils as cvt_utils
import motionblender.lib.dataset as dataset 
import motionblender.lib.animate as anim
import motionblender.lib.mesh2d as mesh_lib
from motionblender.train_final import iPhoneDatasetVideoView
from dataclasses import asdict
from tqdm.auto import tqdm, trange

import open3d as o3d
import torch
import numpy as np
from sklearn.cluster import DBSCAN

import cupy as cp
from cuml import DBSCAN as cuDBSCAN 

from pyvista.plotting.themes import _TrameConfig
_TrameConfig.jupyter_server_port = 8891

def robust_pointcloud_processing(points_tensor, colors_tensor=None):
    """
    Enhanced point cloud processing with:
    1. Statistical outlier removal
    2. Radius outlier removal
    3. DBSCAN clustering to keep only largest cluster
    
    Args:
        points_tensor: PyTorch tensor of shape (N, 3) containing point positions
        colors_tensor: Optional PyTorch tensor of shape (N, 3) containing RGB colors
    
    Returns:
        processed_points: PyTorch tensor of cleaned point positions
        processed_colors: PyTorch tensor of corresponding colors (None if input was None)
    """
    # Convert to Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    points_np = points_tensor.cpu().numpy()
    pcd.points = o3d.utility.Vector3dVector(points_np)
    
    if colors_tensor is not None:
        colors_np = colors_tensor.cpu().numpy()
        pcd.colors = o3d.utility.Vector3dVector(colors_np)

    # 1. First pass: Statistical outlier removal (aggressive)
    _, stat_ind = pcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.5)
    stat_pcd = pcd.select_by_index(stat_ind)

    # 2. Second pass: Radius outlier removal (aggressive)
    _, radius_ind = stat_pcd.remove_radius_outlier(nb_points=30, radius=0.1)
    radius_pcd = stat_pcd.select_by_index(radius_ind)

    # 3. Cluster the remaining points to keep only largest cluster
    points_for_clustering = np.asarray(radius_pcd.points)
    
    # DBSCAN parameters - may need adjustment based on your point cloud density
    labels = cp.asnumpy(cuDBSCAN(eps=0.1, min_samples=50).fit_predict(cp.asarray(points_for_clustering)))
    # labels = clustering.labels_
    
    # Find largest cluster (ignore noise points labeled as -1)
    unique_labels, counts = np.unique(labels[labels >= 0], return_counts=True)
    if len(unique_labels) == 0:
        return torch.empty((0, 3)), torch.empty((0, 3)) if colors_tensor is not None else None
    
    largest_cluster_label = unique_labels[np.argmax(counts)]
    cluster_mask = (labels == largest_cluster_label)
    
    # Get points and colors from largest cluster
    final_points = points_for_clustering[cluster_mask]
    
    if colors_tensor is not None:
        remaining_colors = np.asarray(radius_pcd.colors)[cluster_mask]
    else:
        remaining_colors = None

    # Convert back to PyTorch tensors
    processed_points = torch.from_numpy(final_points).float()
    processed_colors = torch.from_numpy(remaining_colors).float() if remaining_colors is not None else None

    return processed_points, processed_colors

/scratch/xz653/code/shape-of-motion
Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
NumExpr defaulting to 16 threads.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
SCENE = ['cloth', 'microwave'][0]
INSTANCE_ID = 2
data_cfg = dataset.MotionBlenderGeneralDataConfig(data_dir=f'datasets/robot/{SCENE}', split='train', use_tracks=False, normalize_scene=False, use_median_filter=True)
D_center = iPhoneDatasetVideoView(dataset.MotionBlenderGeneralDataset(**asdict(data_cfg), img_prefix='center'))

2025-04-19 15:52:06.759 | WARNING  | motionblender.lib.dataset:__init__:400 - use_tracks is False, but given_cano_t is not set


Loading train images:   0%|          | 0/374 [00:00<?, ?it/s]

2025-04-19 15:52:33.266 | WARNING  | motionblender.lib.dataset:__init__:534 - preinited motion graph found, cano_t = 0


Loading train depths:   0%|          | 0/374 [00:00<?, ?it/s]

max_depth_value tensor(42.8425)


Processing depths:   0%|          | 0/374 [00:00<?, ?it/s]

2025-04-19 15:53:40.567 | WARNING  | motionblender.lib.dataset:__init__:703 - no scene normalization is applied for /scratch/xz653/datasets/mb-corl/v3/microwave!


In [3]:
output_path = f'outputs/tmp/robot/init/{SCENE}.cpkl'

INIT_2D_MESH_COORDS = [[0.73, 0, 0.194], [0.73, 0.27, 0.194], [0.49, 0.27, 0.194]]
INIT_DOUBLE_LINKS = [(0, 1), (0, 2)]
INIT_DOUBLE_LINK_JOINTS = [[0.62, -0.07, 0.254], [0.39, 0.34, 0.254], [0.93, 0.08, 0.254]]

if osp.exists(output_path):
    params = load_cpkl(output_path)
else:
    num_frames = len(D_center)
    require_datasets = [D_center]
    params = {
        'joints': [],
        'rgbs': [],
        'pts': [],
        'links': []
    }
    for i in trange(0, num_frames):
        pts_ls, rgbs_ls = [], []
        for D in require_datasets:
            sample = D[i]
            H, W, _ = sample['imgs'].shape
            pts, rgbs = cvt_utils.get_pointcloud_from_rgb_depth_cam(rgb=sample['imgs'], depths=sample['depths'], 
                                                                    c2w=torch.inverse(sample['w2cs']), 
                                                        X_2d3d=sample['Ks'], image_w=W, image_h=H)
            fg_mask = (sample['instance_masks'] == INSTANCE_ID).flatten()
            pts, rgbs = pts[fg_mask], rgbs[fg_mask]
            pts, rgbs = robust_pointcloud_processing(pts, rgbs)
            pts_ls.append(pts)
            rgbs_ls.append(rgbs)
        pts = torch.cat(pts_ls, dim=0)
        rgbs = torch.cat(rgbs_ls, dim=0)
        params['pts'].append(pts)
        params['rgbs'].append(rgbs)
        if SCENE == 'cloth':
            params['joints'].append(INIT_2D_MESH_COORDS)
        elif SCENE == 'microwave':
            params['joints'].append(INIT_DOUBLE_LINK_JOINTS)
            params['links'] = INIT_DOUBLE_LINKS
        else:
            raise NotImplementedError
    dump_cpkl(output_path, params)

In [ ]:
plotter = pv_editor.Plotter(params)
# pv.global_theme.allow_empty_mesh = True
try:
    plotter.set_params(params)
except NameError:
    plotter = pv_editor.Plotter(params)
    plotter.nxy = (2, 2)

print(plotter.url)
plotter.render() # now go to editor and modify the joint positions

Translator(prefix=None)
awaiting runner setup
awaiting site startup
Print WSLINK_READY_MSG
Schedule auto shutdown with timout 0
awaiting running future
trigger(trigger__1)
trigger(trigger__2)
trigger(P_0x2ade199b3fa0_0Camera)
trigger(P_0x2ade199b3fa0_0AnimateStart)
trigger(P_0x2ade199b3fa0_0AnimateStop)
js_key = class
js_key = style
js_key = fluid
js_key = class
before: class = { 'rounded-circle': !P_0x2ade199b3fa0_0_show_ui }
(prefix=None) token {
has({ => {) = False
(prefix=None) token  
has(  =>  ) = False
(prefix=None) token '
has(' => ') = False
(prefix=None) token rounded
has(rounded => rounded) = False
(prefix=None) token -
has(- => -) = False
(prefix=None) token circle
has(circle => circle) = False
(prefix=None) token '
has(' => ') = False
(prefix=None) token :
has(: => :) = False
(prefix=None) token  
has(  =>  ) = False
(prefix=None) token !
has(! => !) = False
(prefix=None) token P_0x2ade199b3fa0_0_show_ui
has(P_0x2ade199b3fa0_0_show_ui => P_0x2ade199b3fa0_0_show_ui) = True


2025-04-19 15:49:04.468 | INFO     | motionblender.lib.pv_editor:url:414 - pyvista viewer url: http://localhost:8891/index.html?ui=P_0x2ade199b3fa0_0&reconnect=auto


http://localhost:8891/index.html?ui=P_0x2ade199b3fa0_0&reconnect=auto


127.0.0.1 [19/Apr/2025:14:49:05 -0500] "GET /index.html?ui=P_0x2ade199b3fa0_0&reconnect=auto HTTP/1.1" 200 238 "-" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
127.0.0.1 [19/Apr/2025:14:49:05 -0500] "POST /paraview/ HTTP/1.1" 405 210 "http://localhost:8891/index.html?ui=P_0x2ade199b3fa0_0&reconnect=auto" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
client 62bf3071ebd84155843c0248e86cea22 connected


# Export

In [8]:
dump_cpkl(output_path, plotter.params)

In [ ]:
# saving pre init motion graphs
params = plotter.params
print(SCENE, 'pre inited frames', params['pre_inited_frames'])
pre_inited_frames = []

for a in params['pre_inited_frames'].split(','):
    if '-' in a:
        start, end = a.split('-')
        start, end = int(start), int(end)
        pre_inited_frames += list(range(start, end+1))
    else:
        pre_inited_frames.append(int(a))

assert 0 in pre_inited_frames, 'frame 0 should be pre inited'

result = {
    'cano_t': 0,
     'instances': {
        INSTANCE_ID: {}
     }
}

for frame_id in pre_inited_frames:
    if SCENE == 'cloth':
        joints = params['joints'][frame_id ]
        joints, faces, keypoint_inds = mesh_lib.generate_parallelogram_mesh_3d(joints[0], joints[1], joints[2], nx=plotter.nxy[0], ny=plotter.nxy[1])
        links = mesh_lib.faces_to_edges(torch.as_tensor(faces)).tolist()
    else:
        joints = params['joints'][frame_id].tolist()
        links = params['links'].tolist()

    result['instances'][INSTANCE_ID][frame_id] = {
        'joints': torch.as_tensor(joints).tolist(),
        'links': links
    }

dump_json(osp.join(data_cfg.data_dir, 'preinited_motion_graphs.json'), result, indent=4)

microwave pre inited frames 0-12,370-374


: 